# Jupyter notebook for anomaly 3d models

# Pip install librarys

In [ ]:
%pip install trame
%pip install vtk
%pip install fiona
%pip install gemgis
%pip install rasterio
%pip install pyvista
%pip install trimesh
%pip install pyglet
%pip install ipywidgets
%pip install ipyleaflet
%pip install -i https://test.pypi.org/simple/ auscopecat

In [ ]:
import numpy as np
import trimesh
import re

# Read_ts function to read the GoCAD file.

In [ ]:
#TS parsing code is a  modified version of https://github.com/RichardScottOZ/GOCAD_TS-Surface-Reader/blob/main/GOCAD-Reader-example.ipynb
#Thanks for Richard Scott's contribution.
def read_ts(tsfile):
    vrtx = []
    trgl = []
    #split atom line by by space, make vertex at that point in VRTX list
    #go back and get atomlist[1]  VRTXLIST[atomlist[1]-1]

    for line in tsfile:
        if 'VRTX' in line:
            l_input = re.split(r'[\s]+', line)
            temp = np.array(l_input[2:5])
            #temp = np.array(l_input[2:6])
            vrtx.append(temp.astype(float))

        if 'ATOM' in line:
            l_input = re.split(r'[\s]+', line)
            #vertex_id = l_input[1]
            vertex_id_atom = l_input[2]
            #print(len(vrtx), vertex_id, vertex_id_atom)
            vrtx.append(vrtx[ int(vertex_id_atom) -1])

        if 'TRGL' in line:
            l_input = re.split(r'[\s]+', line)
            temp = np.array(l_input[1:4])
            trgl.append(temp.astype(int))

    vrtx = np.asarray(vrtx)
    trgl = np.asarray(trgl)

    crs_dict = {"NAME":None,"AXIS_NAME":None,"AXIS_UNIT":None,"ZPOSITIVE":None}
    check_dist = {"SURFACE":tsfile[-1],"NAME":None,"COORD":0,"TFACE":0,"PVRTX":0,"VRTX":0,"TRGL":0,"ATOM":0,"CRS":crs_dict, "COLOR":None}

    for line in tsfile:
        ##HEADER
        if 'color:' in line:
            strcolor = line.split(':')
            tricolor = re.split(r'[\s]+', strcolor[1])
            tricolor.pop()
            tricolor[0] = float(tricolor[0]) #rgb 0-1 colors R
            tricolor[1] = float(tricolor[1]) #rgb 0-1 colors G
            tricolor[2] = float(tricolor[2]) #rgb 0-1 colors B
            check_dist["COLOR"] = tricolor
        if 'name:' in line:
            strname = line.split(':')
            usename = strname[1].replace("\n",'')
            check_dist["NAME"] = usename
        if 'NAME ' in line and "AXIS" not in line:
            strname = re.split(r'[\s]+', line)
            crs = strname[1].replace("\n",'')
        if 'AXIS_NAME' in line:
            axis_name = re.split(r'[\s]+', line)
            axis_name.pop()
        if 'AXIS_UNIT' in line:
            axis_unit = re.split(r'[\s]+', line)
            axis_unit.pop()
        if 'ZPOSITIVE' in line:
            zpositive = re.split(r'[\s]+', line)[1].replace("\n",'')
        if 'END_ORIGINAL_COORDINATE' in line:
            crs_dict["NAME"] = crs
            crs_dict["AXIS_NAME"] = axis_name
            crs_dict["AXIS_UNIT"] = axis_unit
            crs_dict["ZPOSITIVE"] = zpositive

        ####Data
        if 'TFACE' in line:
            check_dist["TFACE"] += 1
        if 'COORDINATE_SYSTEM' in line:
            check_dist["COORD"] += 1
        if 'ATOM' in line:
            check_dist["ATOM"] += 1
        if 'PVRTX' in line:
            check_dist["PVRTX"] += 1
        if 'VRTX' in line:
            check_dist["VRTX"] += 1
        if 'TRGL' in line:
            check_dist["TRGL"] += 1

    return vrtx, trgl, check_dist

# User input for search

In [ ]:
#Get bbox for search
from io import StringIO
from IPython.display import display, Markdown
from ipyleaflet import basemaps, GeomanDrawControl, Map, CircleMarker
import ipywidgets as widgets
import pandas as pd
from auscopecat.network import request

AUSTRALIA_BBOX = {'north': -10.6681, 'east': 153.5694, 'south': -43.6345, 'west': 113.3389}
#Get anomalyName for search
anomalyName_text = widgets.Text(value='Pilbara', placeholder='the anomaly Name to search.', description='Name to search:')

display(Markdown('## Input a Name and BBox to search'))

australia_btn = widgets.Button(description = 'Select Australia', button_style = '', tooltip = 'Reset bounds to Australia', icon = '')
search_btn = widgets.Button(description = 'Search Anomaly', button_style = '', tooltip = 'Search Anomaly', icon = '')
north_field = widgets.BoundedFloatText(description = 'North', value = AUSTRALIA_BBOX['north'], min = -90.0, max = 90.0, style = {'font_size': '170%' })
west_field = widgets.BoundedFloatText(description = 'West', value = AUSTRALIA_BBOX['west'], min = -180.0, max = 180.0, style = {'font_size': '170%' })
south_field = widgets.BoundedFloatText(description = 'South', value = AUSTRALIA_BBOX['south'], min = -90.0, max = 90.0, style = {'font_size': '170%'})
east_field = widgets.BoundedFloatText(description = 'East', value = AUSTRALIA_BBOX['east'], min = -180.0, max = 180.0, style = {'font_size': '170%'})
input_box_layout = widgets.Layout(display = 'flex', flex_flow = 'row', align_items = 'stretch')
input_box = widgets.Box(children=[anomalyName_text, north_field, west_field, south_field, east_field, australia_btn, search_btn], layout = input_box_layout)

display(Markdown('\n#### User Input Box'))
display(input_box)

defaultLayout=widgets.Layout(width='1280px', height='1024px')
m = Map(basemap=basemaps.Esri.WorldStreetMap, center = (-29.6, 133.0), zoom = 5,layout = defaultLayout)
df = None
def reset_bounds(b):
    north_field.value = AUSTRALIA_BBOX['north']
    south_field.value = AUSTRALIA_BBOX['south']
    west_field.value = AUSTRALIA_BBOX['west']
    east_field.value = AUSTRALIA_BBOX['east']

def search_anomaly(b):
    global df
    #Searching
    bbox = f'{float(west_field.value):.2f}, {float(south_field.value):.2f}, {float(east_field.value):.2f}, {float(north_field.value):.2f}'
    name = f'{anomalyName_text.value}' #'Pilbara'
    cql_filter = ''
    if name :
        if cql_filter.strip() != "":
            cql_filter +=  ' AND '
        cql_filter += f'AnomalyName like \'%{name}%\''

    if bbox :
        if cql_filter.strip() != "":
            cql_filter +=  ' AND '
        cql_filter += f'BBOX(CentreLocation,{bbox})'

    print(f'searching for {cql_filter}')
    # cql_filter = f'AnomalyName like \'%{name}%\''
    url = 'https://remanentanomalies.csiro.au/geoserver/ows'
    params = {
                'service': 'WFS',
                'version': '1.1.0',
                'request': 'GetFeature',
                'typename': 'RemAnom:Anomaly',
                'outputFormat': 'csv',
                'srsname': 'EPSG:4326',
                'CQL_FILTER': cql_filter,
                'maxFeatures': str(10000)
                }
    try:
        response = request(url,params,'POST')
    except Exception as e:
        print(f'Error querying data: {e}',500)
    csvBuffer = StringIO(response.text)
    df = pd.read_csv(filepath_or_buffer = csvBuffer, low_memory=False)
    print(df.shape)
    df.head(10)

    #clean all the markerLayers first
    for layer in m.layers:
        if isinstance(layer, CircleMarker):
            m.remove_layer(layer)
    #add circle-dot for anomalies boreholes:
    for index, row in enumerate(df.itertuples()):
        # if index > 10:
        #     break
        point = row[29]
        found = re.search(r'\((.+?)\)', point)
        if not found:
            continue
        cx, cy = found.group(1).split(' ')
        selectedAnomaly = f'{row[1]}@{row[7]}'
        html = widgets.HTML()
        html.value = selectedAnomaly
        marker = CircleMarker(name = selectedAnomaly, location = (cy,cx),radius = 5, color = 'green', fillcolor='green')
        marker.on_click(create_popup_click(val1 = selectedAnomaly))
        m.add_layer(marker)
        marker.popup = html

australia_btn.on_click(reset_bounds)
search_btn.on_click(search_anomaly)


draw_control = GeomanDrawControl()
draw_control.circlemarker = {}
draw_control.polyline = {}
draw_control.polygon = {}
draw_control.rectangle = {'shapeOptions': {'fillColor': '#fca45d','color': '#fca45d','fillOpacity': 0.7}}

def handle_draw(target, action, geo_json):
    if action == 'create' or action == 'drag':
        coords = geo_json[0]['geometry']['coordinates'][0]
        if len(coords) == 5:
            west_field.value = str(coords[0][0])
            south_field.value = str(coords[0][1])
            east_field.value = str(coords[2][0])
            north_field.value = str(coords[2][1])
            # TODO: Enable Rectangle control (possible?)    
    elif action == 'remove':
        # TODO: Disable Rectangle control (possible?)
        pass
selectedAnomaly =''

def create_popup_click(val1):
    def popup_click(event, type, coordinates, val1 = val1):
        global selectedAnomaly
        selectedAnomaly = str(val1)
    return popup_click

draw_control.on_draw(handle_draw)
m.add(draw_control)
m

In [ ]:
#show the search result in table.df

if df is not None:
    print(df.shape)
else:    
    raise SystemExit('Could not find any anomaly.Please change a name or bbox to seach again!')
df.head(10)

In [ ]:
from PIL import Image
import requests
import matplotlib.pyplot as plt
from zipfile import ZipFile, BadZipFile
from io import BytesIO
import urllib
import os

try:
    anomalyId = selectedAnomaly.split('@')[0].split('.')[1]
except Exception as ex:
    print(str(ex))
    raise SystemExit('Could not get anomalyID. Please click the dot to select one first!')
print(anomalyId)
anomalyJpegUrl = f'https://remanentanomalies.csiro.au/getJpeg.ashx?anomalyId={anomalyId}'
print (anomalyJpegUrl)
anomaly3dModelUrl = f'https://remanentanomalies.csiro.au/getAllModelsForAnomaly.ashx?anomalyid={anomalyId}'
print (anomaly3dModelUrl)

#download anomalyJpeg
anomalyJpeg = Image.open(requests.get(anomalyJpegUrl, stream=True).raw)
plt.imshow(anomalyJpeg)
anomalyJpeg.save(f'magnetic-anomaly-{anomalyId}.jpg')

#download anomaly3dModelUrl
try:
    z = ZipFile(BytesIO(urllib.request.urlopen(anomaly3dModelUrl).read()))
except BadZipFile:
    raise SystemExit(f'Could not find the 3D TS model for anomaly.{anomalyId}.')
        
anomalyModelFileName:str = ''
for name in z.namelist():
    if '.zip' not in name:
        continue
    z2 = ZipFile(BytesIO(z.read(name)))
    for name2 in z2.namelist():
        if '.ts' not in name2:
            continue
        tsFile = z2.read(name2)
        anomalyModelFileName = os.path.basename(name2)
        with open(f'{anomalyModelFileName}',mode='wb') as f:
            f.write(tsFile)

In [ ]:
#Render the TS file by Trimesh
try:
    v, t, meta = read_ts(tsFile.decode("utf-8").splitlines())
    color = meta["COLOR"]

    tri_index = t
    faces = np.empty_like(tri_index, shape=(tri_index.shape[0], 4) )
    faces[:, 0] = 3
    faces[:, 1:] = tri_index    
except Exception as gocadE:
    print(gocadE)   
#t = t -1
mesh = trimesh.Trimesh(vertices=v, faces=t-1, face_colors= color, process=False)
mesh.show()

In [ ]:
#Render the TS file by plt
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

index = 0 
def rotate(angle):
    ax.view_init(azim=angle)
    return   
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(mesh.vertices[:, 0], mesh.vertices[:,1], triangles=mesh.faces, Z=mesh.vertices[:,2],color = color) 
#ax.view_init(-140, 30)
#ani = animation.FuncAnimation(fig, update, interval=5, blit=False,cache_frame_data=True,frames=100)

ani = animation.FuncAnimation(fig, rotate, frames=np.arange(0,362,2),interval=100)
#plt.show()
anomalyGifName = f'magnetic-anomaly-{anomalyId}.gif'
ani.save(anomalyGifName, dpi=100, writer='Pillow')



In [ ]:
print(anomalyGifName)
imgTag = f'<img src=\"{anomalyGifName}\">'
HTML(imgTag)